In [4]:
lst = [[1,2],[2,6],[2,7],[2,3],[3,4],[3,5],[4,5],[8,9],[5,10]]
pyramid = [[1,2],[1,3],[1,4],[2,5],[2,6],[4,7],[4,8], [6,7], [9,10]]
cherries = [[1,2],[1,6],[2,3],[2,5],[3,4],[4,5],[6,9],[6,7],[8,9],[7,8]]

In [5]:
#T1.0 Create the graph dictionary
def graph(lst):
    graph = {}
    for sublist in lst:
        if not sublist[0] in graph.keys():
            graph[sublist[0]] = [sublist[1]]
        else:
            graph[sublist[0]].append(sublist[1])
        if not sublist[1] in graph.keys():
            graph[sublist[1]] = [sublist[0]]
        else:
            graph[sublist[1]].append(sublist[0])    
    return graph
graph(pyramid)

{1: [2, 3, 4],
 2: [1, 5, 6],
 3: [1],
 4: [1, 7, 8],
 5: [2],
 6: [2, 7],
 7: [4, 6],
 8: [4],
 9: [10],
 10: [9]}

In [2]:
#T1.0 Alternative (do not use when debugging, complicated when the dict is not ordered)
def make_graph(lst):
    graph_keys = list(set([x for sublist in lst for x in sublist]))
    graph = {}
    for key in graph_keys:
        graph[key] = list(set([value for sublist in lst for value in sublist if value != key if key in sublist]))
    return graph
make_graph(pyramid)

{1: [2, 3, 4],
 2: [1, 5, 6],
 3: [1],
 4: [8, 1, 7],
 5: [2],
 6: [2, 7],
 7: [4, 6],
 8: [4],
 9: [10],
 10: [9]}

In [14]:
#T1.1 find if two vertices are connected (recursion runs a dfs)
def connected(graph, start, end):
    if end == start:
        return True
    eliminate.append(start)
    for start in graph[start]:
        if not start in eliminate:
            if connected(graph, start, end):
                return True
    return False
eliminate = []
connected(graph(pyramid), 2, 5)

True

In [15]:
#T1.2 Find the connection path (first path found) using recursion
def recursion_dfs_path(graph, start, end):
    if start == end:
        return 'start = end'
    visited.append(start)
    for child in graph[start]:
        if not child in visited:
            if recursion_dfs_path(graph, child, end):
                path.insert(0, child)
                return path
    return path
visited = []
path = []
recursion_dfs_path(graph(pyramid), 1, 8)

[2, 6, 7, 4, 8]

<p>BFS (breadth first traversal) = LIFO = stack => returns the first path found</p>
<p>vs. </p>
<p>DFS (depth first) = FIFO = queue => returns the optimal path by exploring each sub-level superficially instead of all the way down</p>
<p>Recursions can only run DFS, and while loop can return both. Note that while loop dfs will read the children nodes in reverse order compared with recursion</p>

In [8]:
#find_path to build the path dictionary from end to start after the route is found
def find_path(parent, end):
    node = end
    path = []
    while node in parent.keys():
        if not end in path:
            path.append(end)
        path.append(parent[node])
        node = parent[node]    
    return list(reversed(path))

In [9]:
#T1.2 Alternative Find the connection path (first path found) using a while loop
def while_dfs_path(graph, start, end):
    stack = [start]
    visited = [start]
    parent = {}
    while stack:
        current = stack.pop()
        if current == end:
            break
        for child in graph[current]:
            if child not in visited:
                stack.append(child)
                visited.append(child)
                parent[child] = current
    return find_path(parent, end)
start=1
end = 8
while_dfs_path(graph(pyramid), start, end)

[1, 4, 8]

In [10]:
#T1.3 Find the connection path (fastest path)
def while_bfs_path(graph, start, end):
    queue = [start]
    visited = [start]
    parent = {}
    while queue:
        current = queue.pop(0) 
        if current == end:
            break
        for child in graph[current]:
            if child not in visited:
                queue.append(child)
                visited.append(child)
                parent[child] = current
    return find_path(parent, end)
start=1
end = 8
while_bfs_path(graph(pyramid), start, end)

[1, 4, 8]

In [11]:
#1.4 Add weights (create priority queue)
#Create weighted graph

#format for weighted: weight as last item in the list
weighted = [[1,2,1],[1,3,8],[1,4,6],[2,5,4],[2,6,1],[4,7,2],[4,8,5], [6,7,1]]
weighted1 = [[1,2,1], [1,3,5], [1,4,2], [2,5,3], [2,6,2], [3,6,1], [3,7,2], [4,7,5], [4,8,10], [6,7,3]]
weighted2 = [[1,2,10],[1,3,5],[2,3,2],[2,4,5],[3,4,1]]

def weighted_graph(graph_as_list): 
    graph_copy = graph_as_list.copy()
    for sublist in graph_as_list:
        new_sub_list = [sublist[i] for i in [1,0,2]]
        graph_copy.append(new_sub_list)
    
    graph_keys = list(set([x[0] for x in graph_copy]))
    
    graph = {}
    for key in graph_keys:
        graph[key] = sorted([(x[1],x[2]) for x in graph_copy if x[0] == key], key=lambda tup: tup[1])
        
    return graph
weighted_graph(weighted1)

{1: [(2, 1), (4, 2), (3, 5)],
 2: [(1, 1), (6, 2), (5, 3)],
 3: [(6, 1), (7, 2), (1, 5)],
 4: [(1, 2), (7, 5), (8, 10)],
 5: [(2, 3)],
 6: [(3, 1), (2, 2), (7, 3)],
 7: [(3, 2), (6, 3), (4, 5)],
 8: [(4, 10)]}

In [12]:
#Create priority queue (class). Alternative: use the heapq library
class Pqueue():
    """stores nodes with their weights
    updates the weights as the path function goes down the path
    retrieves nodes in the order of their weights"""
    def __init__(self, lst):
        self.pqueue = lst
        
    def pop(self):
        minimum = self.pqueue.index(min(self.pqueue, key = lambda t: t[1]))
        return self.pqueue.pop(minimum)
    
    def append(self, node, weight):
        self.pqueue.append((node, weight))
        
    def query(self, node):
        query = [y for x, y in self.pqueue if x == node]
        return query
        
    def __len__(self):
        return len(self.pqueue)
        
test = Pqueue([])
test.append(5,4)
test.append(2,1)
test.append(4,5)
print(test.pqueue)
print(test.query(5))
print(test.pop())
print(test.pop())
print(test.pop())

[(5, 4), (2, 1), (4, 5)]
[4]
(2, 1)
(5, 4)
(4, 5)


In [13]:
def uniform_cost_search(graph, start, end):
    queue = Pqueue([(start,0)])
    visited = [start]
    parent = {}
    while queue:
        current, pcost = queue.pop()
        if current == end:
            break
        for child, weight in graph[current]:
            path_cost = weight + pcost
            if child in visited and queue.query(child) and queue.query(child)[0] > path_cost:
                queue.append(child, path_cost)
                parent[child] = current
            if child not in visited:
                queue.append(child, path_cost)
                visited.append(child)
                parent[child] = current
    return find_path(parent, end)
start = 2
end = 4

uniform_cost_search(weighted_graph(weighted), start, end)

[2, 6, 7, 4]

In [ ]:
#pqueue class to be improved for efficiency: dictionary + sorted (tree?) + check headq module

In [ ]:
#T1.5 A-Star (bfs using graph weighted with heuristic function)
def a_star_search(start):
    queue = Pqueue([(start,0)])
    visited = [start]
    parent = {}
    counter = 0
    while queue:
        current, pcost = queue.pop()
        if current.is_terminal():
            break
        for child, action in current.children():
            path_cost = child.distance() + pcost
            if child in visited and queue.query(child) and queue.query(child)[0] > path_cost:
                queue.append(child, path_cost)
                parent[child.data] = current.data
            elif child not in visited and child.data != start.data:
                queue.append(child, path_cost)
                visited.append(child)
                parent[child.data] = current.data
    return find_path(parent, current.data)

In [ ]:
#2.0 Add slip and fall situations (10% chance to end up on a child node)
#MDP: also similar concept than queues for slip and fall (Markov Decision Process)